In [78]:
import sympy as sp
import numpy as np
from sympy.physics.quantum import TensorProduct

def get_bias_operator(A0, A1, B0, B1):
    
    n=A0.shape[0]
    # Define the complex exponential factor
    w_n = sp.exp((2*sp.pi*sp.I)/n)
    
    # Initialize the bias operator as a zero matrix in SymPy
    bias_operator = TensorProduct(sp.zeros(n,n),sp.zeros(n,n))
    
    # Loop to compute the bias operator
    for i in range(1, n):
        bias_operator += TensorProduct(A0**i, B0**-i)
        bias_operator += TensorProduct(A0**i, B1**i)
        bias_operator += TensorProduct(A1**i, B0**-i)
        bias_operator += (w_n**(-i)) * TensorProduct(A1**i, B1**i)
    
    return bias_operator

def generate_state(n):
    gamma = sp.sqrt(2*n + 2/(sp.sin(sp.pi/(2*n))))
    z_n = sp.exp(sp.I*sp.pi/(2*n))
    state = TensorProduct(sp.zeros(n,1),sp.zeros(n,1))
    for i in range (0,n):
        state = state + (1-z_n**(n+2*i+1))*(TensorProduct(cycle_permutation(i,n),cycle_permutation(-i,n)))
    state = (1/gamma)*state
    return state

def cycle_permutation(i,n):
    zero_vector = sp.zeros(n,1)
    placement = i%n
    zero_vector[placement, 0] = 1
    return zero_vector

def getz2(n):
    return (sp.exp((sp.I*sp.pi)/(2*n)))**2

def win_probability(bias_operator,state,n):
    x = bias_operator*state
    y = state.H*x
    number = y[0,0].evalf() 
    return (number/(4*n) + 1/n)


In [95]:
A04 = sp.Matrix([[0,0,0,1], [1,0,0,0],[0,1,0,0],[0,0,1,0]])
B04 = sp.Matrix([[0,0,0,1], [1,0,0,0],[0,1,0,0],[0,0,1,0]])
B14 = sp.Matrix([[0,-getz2(4),0,0], [0,0,getz2(4),0],[0,0,0,getz2(4)],[getz2(4),0,0,0]])
A14 = sp.Matrix([[0,0,0,-getz2(4)],[getz2(4),0,0,0],[0,getz2(4),0,0],[0,0,getz2(4),0]])
bell_state = sp.Matrix([1/2,0,0,0,0,1/2,0,0,0,0,1/2,0,0,0,0,1/2])
GHZ = sp.Matrix([1/(2**(1/2)),0,0,0,0,0,0,0,0,0,0,0,0,0,0,1/(2**(1/2))])
W = sp.Matrix([0,1/2,1/2,0,1/2,0,0,0,1/2,0,0,0,0,0,0,0])
rambo = sp.Matrix([1/2,0,0,1j/2,0,0,0,0,0,0,0,0,1/2+(1j/2),0,0,1/2*(np.cos(np.pi/4)+1j*np.sin(np.pi/4))])


win_probability(get_bias_operator(A04,A14,B04,B14),generate_state(4),4)


0.826640741219094

In [81]:
A0 = sp.Matrix([[0,1],[1,0]])
A1 = sp.Matrix([[0,0-1j],[0+1j,0]])

A0_U = sp.Matrix([[sp.sqrt(2)/2, sp.sqrt(2)/2], [-sp.sqrt(2)/2, sp.sqrt(2)/2]])
A1_U = sp.Matrix([[-0.707106781186548*sp.I, -0.707106781186548],[-0.707106781186548, -0.707106781186548*sp.I]])

win_probability(get_bias_operator(A0,A1,A0,A1),generate_state(2),2)


0.853553390593274

In [67]:
optimal_state = sp.Matrix([0.3801932173932526-0.25403698614397946j,0j,0j,0j,0j,0j,0j,0.5290046385320097-0.10522556500522232j,0j,0j,0.5290046385320097+0.10522556500522232j,0j,0j,0.3801932173932526+0.25403698614397946j,0j,0j])
optimal_state

Matrix([
[0.380193217393253 - 0.254036986143979*I],
[                                      0],
[                                      0],
[                                      0],
[                                      0],
[                                      0],
[                                      0],
[ 0.52900463853201 - 0.105225565005222*I],
[                                      0],
[                                      0],
[ 0.52900463853201 + 0.105225565005222*I],
[                                      0],
[                                      0],
[0.380193217393253 + 0.254036986143979*I],
[                                      0],
[                                      0]])

In [106]:
v_0 = get_bias_operator(A04,A14,B04,B14)*generate_state(4)

In [98]:
((4+(2/sp.sin(sp.pi/8)))*generate_state(4)).evalf()

Matrix([
[3.50775837894588 - 2.34380921579406*I],
[                                    0],
[                                    0],
[                                    0],
[                                    0],
[                                    0],
[                                    0],
[4.88073002994299 - 0.97083756479695*I],
[                                    0],
[                                    0],
[4.88073002994299 + 0.97083756479695*I],
[                                    0],
[                                    0],
[3.50775837894588 + 2.34380921579406*I],
[                                    0],
[                                    0]])

In [104]:
((get_bias_operator(A04,A14,B04,B14)*generate_state(4)) - ((4+(2/sp.sin(sp.pi/8)))*generate_state(4))).evalf()

Matrix([
[-0.e-136 + 0.e-136*I],
[                   0],
[                   0],
[                   0],
[                   0],
[                   0],
[                   0],
[-0.e-139 + 0.e-140*I],
[                   0],
[                   0],
[-0.e-139 - 0.e-140*I],
[                   0],
[                   0],
[-0.e-136 - 0.e-136*I],
[                   0],
[                   0]])

In [109]:
(((dagger(generate_state(4)))*v_0).evalf())

Matrix([[9.22625185950551]])

In [1]:
A03 = sp.Matrix([[0,0,1],[1,0,0],[0,1,0]])
B03 = sp.Matrix([[0,0,1],[1,0,0],[0,1,0]])
A13 = sp.Matrix([[0,0,-getz2(3)],[getz2(3),0,0],[0,getz2(3),0]])
B13 = sp.Matrix([[0,-getz2(3),0],[0,0,getz2(3)],[getz2(3),0,0]])


NameError: name 'sp' is not defined